### 3rd_ML100Marathon Midterm

- 安隆公司曾是一間能源公司，2001 年破產前是世界上最大的電力、天然氣及電信公司之一。擁有上千億資產的公司於 2002 年竟然在短短幾周內宣告破產，才揭露其財報在多年以來均是造假的醜聞。在本資料集中你將會扮演偵探的角色，透過高層經理人內部的 mail 來往的情報以及薪資、股票等財務特徵，訓練出一個機器學習模型來幫忙你找到可疑的詐欺犯罪者是誰! 我們已經先幫你找到幾位犯罪者 (Person-of-Interest, poi) 與清白的員工，請利用這些訓練資料來訓練屬於自己的詐欺犯機器學習模型吧!

### 特徵說明
- 有關財務的特徵: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (單位皆為美元)。更詳細的特徵說明請參考 enron61702insiderpay.pdf 的最後一頁(請至Data頁面參考該PDF檔)
- 有關 email 的特徵: ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (除了 email_address，其餘皆為次數)

- 嫌疑人的標記，也就是我們常用的 **y**。POI label: [‘poi’] (boolean, represented as integer)

我們也建議你對既有特徵進行一些特徵工程如 rescale, transform ，也試著發揮想像力與創意，建立一些可以幫助找到嫌疑犯的特徵，增進模型的預測能力


In [252]:
# Import 需要的套件
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.preprocessing import scale 
from sklearn.preprocessing import Imputer 
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

%matplotlib inline

### 讀取資料

In [263]:
df_train = pd.read_csv("train_data.csv")
df_test = pd.read_csv("test_features.csv")
submit = pd.read_csv("sample_submission.csv")

In [254]:
print(df_train.info())
print("--------------------------------------------")
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 22 columns):
name                         113 non-null object
bonus                        61 non-null float64
deferral_payments            28 non-null float64
deferred_income              34 non-null float64
director_fees                13 non-null float64
email_address                83 non-null object
exercised_stock_options      81 non-null float64
expenses                     73 non-null float64
from_messages                65 non-null float64
from_poi_to_this_person      65 non-null float64
from_this_person_to_poi      65 non-null float64
loan_advances                2 non-null float64
long_term_incentive          49 non-null float64
other                        69 non-null float64
poi                          113 non-null bool
restricted_stock             82 non-null float64
restricted_stock_deferred    10 non-null float64
salary                       73 non-null float64
shared_receipt_wi

### 前處理


In [264]:
df = df_train.append(df_test) 
df.reset_index(inplace=True, drop = True)
df = df.replace('NaN', np.nan)
######重新整理排列，並歸類類別
payment_data = ['salary',
                'bonus',
                'long_term_incentive',
                'deferred_income',
                'deferral_payments',
                'loan_advances',
                'other',
                'expenses',                
                'director_fees', 
                'total_payments']
stock_data = ['exercised_stock_options',
              'restricted_stock',
              'restricted_stock_deferred',
              'total_stock_value']

email_data = ['to_messages',
              'from_messages',
              'from_poi_to_this_person',
              'from_this_person_to_poi',
              'shared_receipt_with_poi']

features_list =['poi'] + payment_data + stock_data + email_data
#name_features_list = name + ['poi'] + payment_data + stock_data + email_data

df = df[features_list]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 20 columns):
poi                          113 non-null object
salary                       95 non-null float64
bonus                        82 non-null float64
long_term_incentive          66 non-null float64
deferred_income              49 non-null float64
deferral_payments            39 non-null float64
loan_advances                4 non-null float64
other                        93 non-null float64
expenses                     95 non-null float64
director_fees                17 non-null float64
total_payments               125 non-null float64
exercised_stock_options      102 non-null float64
restricted_stock             110 non-null float64
restricted_stock_deferred    18 non-null float64
total_stock_value            126 non-null float64
to_messages                  86 non-null float64
from_messages                86 non-null float64
from_poi_to_this_person      86 non-null float64
from_this_

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [220]:
df['poi'].unique()

array([True, False, nan], dtype=object)

In [221]:
df.describe()

,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,exercised_stock_options,restricted_stock,restricted_stock_deferred,total_stock_value,to_messages,from_messages,from_poi_to_this_person,from_this_person_to_poi,shared_receipt_with_poi
count,1.060000e+02,9.400000e+01,8.200000e+01,6.700000e+01,6.100000e+01,3.500000e+01,1.020000e+02,1.060000e+02,4.600000e+01,1.290000e+02,1.140000e+02,1.150000e+02,4.300000e+01,1.310000e+02,98.000000,98.000000,98.000000,98.000000,98.000000
mean,5.038534e+05,2.071141e+06,1.183462e+06,-8.340788e+05,1.050234e+06,4.795714e+06,8.379710e+05,9.744573e+04,6.164528e+04,4.923960e+06,5.356838e+06,2.220796e+06,6.966023e+04,6.515409e+06,2042.705539,569.930029,64.931487,41.664723,1201.137026
std,2.575914e+06,1.002996e+07,5.355748e+06,3.470459e+06,4.184264e+06,1.947014e+07,4.387838e+06,5.059129e+05,2.073905e+05,2.861769e+07,2.942435e+07,1.224991e+07,2.674319e+06,3.822345e+07,2419.126910,1726.566643,81.421582,93.685948,1105.022610
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.799289e+07,-1.025000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.604490e+06,-7.576788e+06,-4.409300e+04,57.000000,12.000000,0.000000,0.000000,2.000000
25%,1.889435e+05,3.250000e+05,1.062500e+05,-3.503645e+05,0.000000e+00,0.000000e+00,8.020000e+02,1.238600e+04,0.000000e+00,3.199410e+05,3.505130e+05,2.086595e+05,-1.102915e+05,4.152485e+05,623.000000,26.250000,12.000000,1.000000,383.000000
50%,2.483460e+05,7.000000e+05,3.426745e+05,-7.500000e+04,7.312200e+04,0.000000e+00,1.215550e+04,3.786550e+04,0.000000e+00,1.092663e+06,9.132440e+05,4.410960e+05,0.000000e+00,1.038185e+06,1564.500000,50.500000,40.500000,13.500000,887.000000
75%,3.002298e+05,1.100000e+06,6.948620e+05,0.000000e+00,6.495840e+05,0.000000e+00,3.061778e+05,7.360350e+04,1.006335e+05,2.081796e+06,2.266645e+06,9.773805e+05,0.000000e+00,2.554413e+06,2449.750000,291.428571,65.190476,44.761905,1601.250000
max,2.670423e+07,9.734362e+07,4.852193e+07,0.000000e+00,3.208340e+07,8.392500e+07,4.266759e+07,5.235198e+06,1.398517e+06,3.098866e+08,3.117640e+08,1.303223e+08,1.545629e+07,4.345095e+08,15149.000000,14368.000000,528.000000,609.000000,5521.000000


- 從上述分析結果發現共有146筆資料，其中113筆為Train，33筆為Test
- 'total_payments'、'to_messages'有離群值不合理
- 在財務數據方面，從官方文檔中得知，NaN的值表示0，而不是未知数量
- 在電子郵件訊息方面NaN則為未知，to_messages'有NaN值的部分採平均填滿


# 補上缺漏值

In [266]:
#檢查 DataFrame 空缺值的狀態
def na_check(df):
    df_na = (df.isnull().sum() / len(df)) * 100
    df_na = df_na.drop(df_na[df_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :df_na})
    display(missing_data.head(10))
na_check(df)

,Missing Ratio
poi,22.60274


In [265]:
#在財務數據方面，從官方文檔中得知，NaN的值表示0，而不是未知数量
df[payment_data] = df[payment_data].fillna(0)
df[stock_data] = df[stock_data].fillna(0)
#在email方面取該欄位平均後填滿
imp = Imputer(missing_values="NaN" , strategy= "mean" , axis = 0)
#取出poi分類
df_poi = df[df['poi'] == True]
df_nonpoi = df[df['poi']==False]
df_test = df[113:]
#缺失值的处理方式是使用X中的均值（axis=0表示按列进行）代替Y中的缺失值
df_poi.ix[:, email_data] = imp.fit_transform(df_poi.ix[:,email_data])
df_nonpoi.ix[:, email_data] = imp.fit_transform(df_nonpoi.ix[:,email_data])
df_test.ix[:, email_data] = imp.fit_transform(df_test.ix[:,email_data])

df = df_poi.append(df_nonpoi)
df = df.append(df_test)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_

# 檢查離群值
- 檢查異常值/離群值，將與所有支出的相關列加總與個人總支付額相比
- 股票處理亦相同

In [267]:
error = (df[df[payment_data[:-1]].sum(axis = 'columns') != df['total_payments']])

correct = (df[df[payment_data[:-1]].sum(axis = 'columns') == df['total_payments']]).head(5)


error


,poi,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,exercised_stock_options,restricted_stock,restricted_stock_deferred,total_stock_value,to_messages,from_messages,from_poi_to_this_person,from_this_person_to_poi,shared_receipt_with_poi
50,False,0.0,0.0,0.0,0.0,-102500.0,0.0,0.0,0.0,3285.0,102500.0,3285.0,0.0,44093.0,-44093.0,2181.839286,815.928571,60.125,43.517857,1055.5
139,NaN,0.0,0.0,0.0,0.0,0.0,0.0,137864.0,0.0,137864.0,15456290.0,2604490.0,-2604490.0,15456290.0,0.0,523.000000,29.000000,0.000,1.000000,463.0


In [204]:
correct

,poi,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,exercised_stock_options,restricted_stock,restricted_stock_deferred,total_stock_value,to_messages,from_messages,from_poi_to_this_person,from_this_person_to_poi,shared_receipt_with_poi
0,True,420636.0,1750000.0,1617011.0,-3504386.0,0.0,0.0,174839.0,46950.0,0.0,505050.0,19794175.0,2748364.0,0.0,22542539.0,905.000000,18.000000,42.000000,4.000000,864.0
1,True,1111258.0,5600000.0,1920000.0,0.0,0.0,0.0,22122.0,29336.0,0.0,8682716.0,19250000.0,6843672.0,0.0,26093672.0,3627.000000,108.000000,88.000000,30.000000,2042.0
2,True,211844.0,200000.0,0.0,-4167.0,0.0,0.0,1573324.0,22884.0,0.0,2003885.0,1624396.0,869220.0,0.0,2493616.0,225.000000,39.000000,13.000000,14.000000,91.0
3,True,224305.0,800000.0,602671.0,0.0,0.0,0.0,907502.0,118134.0,0.0,2652612.0,0.0,985032.0,0.0,985032.0,1995.666667,60.444444,93.888889,18.777778,1459.0
4,True,240189.0,1250000.0,375304.0,-262500.0,0.0,0.0,486.0,35818.0,0.0,1639297.0,0.0,126027.0,0.0,126027.0,2598.000000,144.000000,199.000000,25.000000,2188.0


In [268]:
#修正第50筆資料
index_50 = df.ix[50, 1:15].tolist()
#刪除第一個欄位向左移動後，如財務數據所示
index_50.pop(0)
index_50.append(0)
#重新補回該數據
df.ix[50, 1:15] = index_50

#修正第139筆資料
index_139 = df.ix[139, 1:15].tolist()
#刪除第一個欄位向右移動後，如財務數據所示
index_139.pop(-1)
index_139 = [0] + index_139
#重新補回該數據
df.ix[139, 1:15] = index_139


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:961: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return getattr(section, self.name)[new_key]
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  import sys
C:\Users\User\Anaconda3\lib\site-pack

In [269]:
#確認財務數據
len(df[df[payment_data[:-1]].sum(axis='columns') != df['total_payments']])

0

In [270]:
#確認股票數據
len(df[df[stock_data[:-1]].sum(axis='columns') != df['total_stock_value']])

0

- 由於數據集太小，異常值影響會非常大

$$ low\ outlier < first\ quartile - 1.5\ x\ IQR $$  
$$ high\ outlier > third\ quartile + 1.5\ x\ IQR $$ 

In [271]:
#保守的刪除異常值
IQR = df.quantile(q=0.75) - df.quantile(q=0.25)

first_quartile = df.quantile(q=0.25) 
third_quartile = df.quantile(q=0.75) 

outliers = df[(df>(third_quartile + 1.5*IQR) ) | (df<(first_quartile - 1.5*IQR) )].count(axis=1) 
outliers.sort_values(axis=0, ascending=False, inplace=True) 
outliers.head(12)

133    14
8      12
119    12
58      8
113     8
33      8
87      7
1       7
47      7
26      5
68      5
63      5
dtype: int64

In [238]:
#統計最後POI的人數
df['poi'].value_counts()

False    101
True      13
Name: poi, dtype: int64

In [239]:
df.isnull().sum().sum()

33

- 然而雖然有許多的離群值，但可能表示某人正在透過大量的洗錢給同夥

# 特徵工程
### 透過mail建立欄位

In [272]:
df['to_poi_ratio'] = df['from_poi_to_this_person'] / df['to_messages']
df['from_poi_ratio'] = df['from_this_person_to_poi'] / df['from_messages']
df['shared_poi_ratio'] = df['shared_receipt_with_poi'] / df['to_messages']

In [273]:
features_list.append('to_poi_ratio') 
features_list.append('from_poi_ratio') 
features_list.append('shared_poi_ratio')
name_features_list.append('to_poi_ratio')
name_features_list.append('from_poi_ratio')
name_features_list.append('shared_poi_ratio')

### 透過財務數據建立欄位
- 因為獎金可能包含犯罪者的利益分配

In [274]:
df['bonus_to_salary'] = df['bonus'] / df['salary']
df['bonus_to_total'] = df['bonus'] / df['total_payments']  

In [275]:
features_list.append('bonus_to_salary')
features_list.append('bonus_to_total')
name_features_list.append('bonus_to_salary')
name_features_list.append('bonus_to_total')

In [276]:
#建立副本，取整數
scaled_df = df.copy()
scaled_df.ix[:,1:] = scale(scaled_df.ix[:,1:])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/us

In [277]:
df2 = df[features_list]

# 套入模型

In [278]:
Train = df2[pd.notnull(df2['poi'])]
Test = df2[~pd.notnull(df2['poi'])]

In [325]:
Test2 = Test.drop(['poi'], axis = 1)

In [284]:
Train['poi'] = Train['poi'].astype('category').cat.codes 

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [322]:
Train.fillna(value= 0, inplace=True) 
Test.fillna(value= 0, inplace=True) 

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:4259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [300]:
#隨機森林樹

rf = RandomForestClassifier(criterion='entropy',
                            n_estimators=50,
                            min_samples_split=5,
                            min_samples_leaf=2,
                            oob_score=True, 
                            random_state=1,
                            n_jobs=-1) 

rf.fit(Train.iloc[:, 1:], Train.iloc[:, 0])
print("%.4f" % rf.oob_score_)

0.9115


In [291]:
# 設定要訓練的超參數組合
param_grid = {"criterion" : ["entropy", "gini"],
              "max_depth" : [4,6,8,10],
              "min_samples_split" : [2,5,10],
              "min_samples_leaf" : [2,5,10],
              "n_estimators" : [20,50, 100], # 使用 n 顆樹
              "max_features" : ['auto', 'sqrt'], # 如何選取 features
              "oob_score" : [True]
             }

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(estimator = RandomForestClassifier(random_state = 50),
                  param_grid = param_grid,
                  cv = 5,
                  scoring = "accuracy")

# 開始搜尋最佳參數
grid_result = grid_search.fit(Train.iloc[:, 1:], Train.iloc[:, 0])




C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [292]:
# 印出最佳結果與最佳參數 
print("best_score  : %s" % grid_result.best_score_) 
print("best_params : %s \n" % grid_result.best_params_)

best_score  : 0.9292035398230089
best_params : {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50, 'oob_score': True} 



In [293]:
rf = grid_result.best_score_

In [294]:
# 訓練模型 
rf.fit(Train.iloc[:, 1:], Train.iloc[:, 0]) 

AttributeError: 'numpy.float64' object has no attribute 'fit'

In [326]:
# 隨機森林預測檔 
rf_res = rf.predict(Test2) 
submit['poi'] = rf_res 
submit['poi'] = submit['poi'].astype(int) 
submit.to_csv('mid_submit.csv', index = False)